In [13]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col,count,hour,lead,when
from pyspark.sql.window import Window


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

In [3]:
EmployeeShifts = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/credit_card") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "EmployeeShifts") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

EmployeeShifts.show()

+-----------+-------------------+-------------------+
|employee_id|         start_time|           end_time|
+-----------+-------------------+-------------------+
|          1|1970-01-01 08:00:00|1970-01-01 12:00:00|
|          1|1970-01-01 11:00:00|1970-01-01 15:00:00|
|          1|1970-01-01 14:00:00|1970-01-01 18:00:00|
|          2|1970-01-01 09:00:00|1970-01-01 17:00:00|
|          2|1970-01-01 16:00:00|1970-01-01 20:00:00|
|          3|1970-01-01 10:00:00|1970-01-01 12:00:00|
|          3|1970-01-01 13:00:00|1970-01-01 15:00:00|
|          3|1970-01-01 16:00:00|1970-01-01 18:00:00|
|          4|1970-01-01 08:00:00|1970-01-01 10:00:00|
|          4|1970-01-01 09:00:00|1970-01-01 11:00:00|
+-----------+-------------------+-------------------+



In [15]:
EmployeeShifts\
        .withColumn("start_hour",hour(col("start_time")))\
        .withColumn("end_hour",hour(col("end_time")))\
        .withColumn("next_start_hour",lead("start_hour").over(Window.partitionBy(col("employee_id")).orderBy("start_time")))\
        .withColumn("fall_back",when(col("next_start_hour").between(col("start_hour"),col("end_hour")),'Yes').otherwise('No'))\
        .filter(col("fall_back")=='Yes')\
        .groupby(col("employee_id"))\
        .count()\
        .show()

+-----------+-----+
|employee_id|count|
+-----------+-----+
|          1|    2|
|          2|    1|
|          4|    1|
+-----------+-----+

